In [19]:
from __future__ import division
import numpy as np
import scipy.sparse as sp
import pandas as pd

In [20]:
web_graph = pd.read_csv('../input/web-Google.txt', sep='\t', comment='#', header=None)

In [21]:
orig_id_to_matrix_id = {}
matrix_id_to_orig_id = []
for num in web_graph[0].values:
    if num not in orig_id_to_matrix_id:
        matrix_id_to_orig_id.append(num)
        orig_id_to_matrix_id[num] = len(matrix_id_to_orig_id) - 1
for num in web_graph[1].values:
    if num not in orig_id_to_matrix_id:
        matrix_id_to_orig_id.append(num)
        orig_id_to_matrix_id[num] = len(matrix_id_to_orig_id) - 1

In [22]:
num_pages = len(matrix_id_to_orig_id)

In [23]:
web_graph['probs'] = 1.0 / web_graph.groupby([0])[1].transform('count')

In [24]:
M = sp.coo_matrix((web_graph.probs.values, 
                   ([orig_id_to_matrix_id[id] for id in web_graph[1].values], 
                    [orig_id_to_matrix_id[id] for id in web_graph[0].values])), 
                  shape=(num_pages, num_pages))

In [25]:
v = np.ones((num_pages, 1)) / num_pages
v_prev = None
for i in xrange(300):
    v = .8 * M.dot(v)
    v = v + (1 - np.sum(v)) / len(v)
    if v_prev is not None:
        sum_abs_diff = np.sum(np.abs(v_prev - v))
        if sum_abs_diff < 1e-10:
            break
    v_prev = v
page_ranks = {}
for i in xrange(len(v)):
    page_ranks[matrix_id_to_orig_id[i]] =  v[i]